% DES431 Project 2: Recommendation System

# Background

**MovieLens** is a movie recommendation system operated by GroupLens, a research group at the University of Minnesota. 

# Task

1. Propose and implement your own recommendation system based on the MovieLens dataset. Use `ratings_train.csv` as the training set, `ratings_valid.csv` as the validation set. Your system may use information from `movies.csv` and `tags.csv` to conduct recommendations. The undisclosed test set will be used to evaluate your system.
   - The data file structure is available at https://files.grouplens.org/datasets/movielens/ml-latest-small-README.html. 
   - The main goal of the recommendation system is to minimize the root-mean-square error.
   - The implementation should include a function named `predict_rating`. This function accepts a DataFrame with two columns `userId` and `movieId`. Then, the function adds a column named `rating` storing a predicted rating of a `movieId` by a `userId`.
   - Your program must return a root-mean-square error value when the validation set is changed to another file. Otherwise, your score will be deducted by 50%.
   - You must modify the given program to make better recommendations. Submitting the original program without modification is considered plagiarism.
2. Prepare slides for a 7-minute presentation to explain your proposed technique and algorithm to conduct recommendation, and show your RMSE results on the validation set.
3. Submit all required documents by April 30, 2023; 23:59. Late submission will not be accepted and will be marked 0. Do not wait until the last minute. Plagiarism and code duplication will be checked. 
4. Present your work on May 1, 2023 within 7 minutes. Exceeding 7 minutes will be subject to point deduction.

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
import imdb
import requests
import tmdbv3api as tmdb
from surprise import Dataset, Reader, SVD, accuracy

# Loading data

In [21]:
ratings_train = pd.read_csv('ratings_train.csv')
ratings_valid = pd.read_csv('ratings_valid.csv')
movies = pd.read_csv('movies.csv')
tags = pd.read_csv('tags.csv',usecols=["userId","movieId","tag"])
links = pd.read_csv('links.csv')

In [22]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [23]:
tags

,userId,movieId,tag
0,2,60756,funny
1,2,60756,Highly quotable
2,2,60756,will ferrell
3,2,89774,Boxing story
4,2,89774,MMA
...,...,...,...
3678,606,7382,for katie
3679,606,7936,austere
3680,610,3265,gun fu
3681,610,3265,heroic bloodshed


In [24]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [25]:
ratings_train

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
96459,610,166534,4.0,1493848402
96460,610,168248,5.0,1493850091
96461,610,168250,5.0,1494273047
96462,610,168252,5.0,1493846352


In [26]:
ratings_valid

,userId,movieId,rating,timestamp
0,4,45,3.0,986935047
1,4,52,3.0,964622786
2,4,58,3.0,964538444
3,4,222,1.0,945629040
4,4,247,3.0,986848894
...,...,...,...,...
2349,561,139385,3.5,1491092337
2350,561,146656,3.5,1491095479
2351,561,149406,3.5,1491091520
2352,561,160438,2.0,1491091498


In [27]:
print("Number of users = "+str(ratings_train["userId"].nunique()))
print("Number of movies = "+str(ratings_train["movieId"].nunique()))

Number of users = 610
Number of movies = 9690


# CONTENT-BASED

In [28]:
m_df=ratings_train[["userId","movieId","rating"]].merge(movies,on="movieId")
m_df

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...
96459,610,160341,2.5,Bloodmoon (1997),Action|Thriller
96460,610,160527,4.5,Sympathy for the Underdog (1971),Action|Crime|Drama
96461,610,160836,3.0,Hazard (2005),Action|Drama|Thriller
96462,610,163937,3.5,Blair Witch (2016),Horror|Thriller


In [29]:
ratings = ratings_train.copy()

# convert UserID, MovieID to categorical type
ratings['userId'] = ratings['userId'].astype('category')
ratings['movieId'] = ratings['movieId'].astype('category')
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
96459,610,166534,4.0,1493848402
96460,610,168248,5.0,1493850091
96461,610,168250,5.0,1494273047
96462,610,168252,5.0,1493846352


In [30]:
# convert genres to Array
m_genre=m_df.copy()
m_genre["genres"]=m_genre["genres"].str.split("|")
# convert array elements to string
m_genre["genres"]=m_genre["genres"].fillna("").astype("str")
m_genre


,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy..."
1,5,1,4.0,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy..."
2,7,1,4.5,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy..."
3,15,1,2.5,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy..."
4,17,1,4.5,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy..."
...,...,...,...,...,...
96459,610,160341,2.5,Bloodmoon (1997),"['Action', 'Thriller']"
96460,610,160527,4.5,Sympathy for the Underdog (1971),"['Action', 'Crime', 'Drama']"
96461,610,160836,3.0,Hazard (2005),"['Action', 'Drama', 'Thriller']"
96462,610,163937,3.5,Blair Witch (2016),"['Horror', 'Thriller']"


In [31]:
# get movie title Series
titles = movies['title']
title_S = pd.Series(movies.index, index=movies['title'])
title_S

title
Toy Story (1995)                                0
Jumanji (1995)                                  1
Grumpier Old Men (1995)                         2
Waiting to Exhale (1995)                        3
Father of the Bride Part II (1995)              4
                                             ... 
Black Butler: Book of the Atlantic (2017)    9737
No Game No Life: Zero (2017)                 9738
Flint (2017)                                 9739
Bungo Stray Dogs: Dead Apple (2018)          9740
Andrew Dice Clay: Dice Rules (1991)          9741
Length: 9742, dtype: int64

In [32]:
# get genre name list
genre_name = set()
for genre in movies['genres'].str.split('|').values:
    genre_name = genre_name.union(set(genre))
genre_name

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

# term-frequency inverse document frequency matrix

In [33]:
# find tf-idf score of genre for  each movie
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 1),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(m_genre['genres'])
print(tfidf_matrix[:10, :10].todense())

[[0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.

In [34]:
# get movie title and rating from IMDB
linksIMDBID=links.copy()
linksIMDBID['imdbId'] = links['imdbId'].astype(str)

def get_movie_title_and_rating(links):
    def retrieve_info(imdbId):
        try:
            movie = ia.get_movie(imdbId)
            title = movie['title']
            rating = movie['rating']
            return title, rating
        except:
            return None, None
        
    # Apply the retrieve_info function to the imdbId column of the link_df
    movie_info = links['imdbId'].apply(retrieve_info)
    
    # Unpack the tuples into separate Series for title and rating
    movie_titles, movie_ratings = zip(*movie_info)
    
    # Return a DataFrame with the movie titles and ratings
    return pd.DataFrame({'movieId': links['movieId'], 'title': movie_titles, 'rating': movie_ratings})

# Apply the get_movie_title_and_rating function to the link DataFrame
movie_info_df = get_movie_title_and_rating(links)

In [35]:
# Get the indices of the movies in the tfidf_matrix
movie_indices = movies.index.tolist()

# Create the tfidf array only for the movies in the movies DataFrame
tfidf_array = tfidf_matrix[movie_indices].toarray()

#Create a new dupe for movies
movies_tf = movies.copy()
# Add the tfidf column to the movies DataFrame
movies_tf['Genre_tfidf'] = pd.Series(tfidf_array.tolist(), index=movies.index)
movies_tf


,movieId,title,genres,Genre_tfidf
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[0.0, 0.363560332296821, 0.5499489959184152, 0..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[0.0, 0.363560332296821, 0.5499489959184152, 0..."
2,3,Grumpier Old Men (1995),Comedy|Romance,"[0.0, 0.363560332296821, 0.5499489959184152, 0..."
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[0.0, 0.363560332296821, 0.5499489959184152, 0..."
4,5,Father of the Bride Part II (1995),Comedy,"[0.0, 0.363560332296821, 0.5499489959184152, 0..."
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,"[0.44828009534257585, 0.0, 0.0, 0.0, 0.4001098..."
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,"[0.44828009534257585, 0.0, 0.0, 0.0, 0.4001098..."
9739,193585,Flint (2017),Drama,"[0.44828009534257585, 0.0, 0.0, 0.0, 0.4001098..."
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,"[0.44828009534257585, 0.0, 0.0, 0.0, 0.4001098..."


# Singular Value Decomposition (SVD)

In [36]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import SVD

# Load ratings dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Load Train dataset
ds_user_item=Dataset.load_from_df(ratings_train[["userId","movieId","rating"]], reader)
# Load Valid dataset (must be changeable)
ds_user_item_valid=Dataset.load_from_df(ratings_valid[["userId","movieId","rating"]], reader)

trainset = ds_user_item.build_full_trainset()
testset= ds_user_item_valid.build_full_trainset()

# Train the SVD model on the training set
svd = SVD(n_factors=100, n_epochs=100, lr_all=0.005, reg_all=0.02)
svd.fit(trainset)

# Evaluate the model on the validation set
predictions = svd.test(testset.build_testset())

# Compute and print the RMSE score for the validation set
accuracy.rmse(predictions)

# Print the estimated rating for each prediction
for prediction in predictions:
    print(prediction.est)


RMSE: 0.8606
3.8499676689040347
2.980287932011277
4.184477208142591
2.458885224231066
4.14579069011846
3.8309629698975836
2.8492123916650494
2.9419216501394816
3.3184363421860867
3.944733019497929
3.2096633749057633
4.847780340365063
4.0972711096711825
4.178986965812479
4.103952757854272
3.4312251429285645
4.191671900583103
4.4026403795033
4.06815592899037
3.5645898827413984
4.012221324546291
4.071730188338541
3.1215939163331106
3.305427333997249
3.819513550100978
3.485146451340631
3.666567624971344
3.410738025594948
2.6026682420696017
3.9899555236350976
4.339681054900847
3.05138725950232
2.254487483861052
4.14336122404567
3.8140334467440016
2.9451638531695523
3.850947735787552
3.331809381776381
3.8570244484691516
2.8485540421469637
3.6855985919447125
2.4473416046551772
3.545705721638312
2.987507562677362
3.5078964597202162
4.277484671150216
3.5650368191541006
3.9317885089783164
4.09576753928996
3.2580690221248
3.7241885731277367
4.165696776588785
3.5322236519015444
2.5027184636209903


In [37]:
uid = [p.uid for p in predictions]
iid = [p.iid for p in predictions]
est = [p.est for p in predictions]
predictions_df = pd.DataFrame({'userId': uid, 'movieId': iid, 'rating': est})
predictions_df

,userId,movieId,rating
0,4,45,3.849968
1,4,52,2.980288
2,4,58,4.184477
3,4,222,2.458885
4,4,247,4.145791
...,...,...,...
2349,561,139385,3.729793
2350,561,146656,3.262620
2351,561,149406,3.723373
2352,561,160438,3.154893


# Constructing model and predicting ratings

In [38]:
# Model construction
avg_rating = ratings_train[['movieId', 'rating']].groupby(by='movieId').mean()
	    
# Prediction
def predict_rating(df):
    # Input: 
	# 	df = a dataframe with two columns: userId, movieId
	# Output:
	#   a dataframe with three columns: userId, movieId, rating
	
   # new_df=pd.merge(df,predictions_df,on=["userId","movieId"],how="left")
    new_df=pd.merge(df,predictions_df,on=["userId","movieId"],how="left")
    return new_df  #df.join(avg_rating, on='movieId')

In [39]:
# Prepare df for prediction
r = ratings_valid[['userId', 'movieId']]

# Predict ratings
ratings_pred = predict_rating(r)
ratings_pred

,userId,movieId,rating
0,4,45,3.849968
1,4,52,2.980288
2,4,58,4.184477
3,4,222,2.458885
4,4,247,4.145791
...,...,...,...
2349,561,139385,3.729793
2350,561,146656,3.262620
2351,561,149406,3.723373
2352,561,160438,3.154893


In [40]:
from sklearn.metrics import mean_squared_error

r_true = ratings_valid['rating'].to_numpy()
r_pred = ratings_pred['rating'].to_numpy()

rmse = mean_squared_error(r_true, r_pred, squared=False)
print(f"RMSE = {rmse:.4f}")

RMSE = 0.8606
